In [4]:
pip install -q transformers matplotlib wordcloud seaborn scikit-plot contractions nltk regex tweetnlp

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Importing Libraries
import pandas as pd
import numpy as np
import warnings
import snscrape.modules.twitter as sntwitter
import requests
import tweetnlp

import seaborn as sns
import matplotlib.pyplot as plt
import scikitplot as skplt
import seaborn as sns
import contractions
import re
import time
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

C:\Users\temit\anaconda3\envs\myenv2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\temit\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\temit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\temit\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
import time
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer
from datetime import datetime, timedelta

In [ ]:
TExtract = pd.read_csv('test_samples.csv',encoding= 'MacRoman')

In [ ]:
TExtract = TExtract.drop('Unnamed: 0', axis=1)


In [ ]:
TExtract = TExtract.rename(columns={'label': 'HumanSentiment'})

In [ ]:
TExtract.info()

In [ ]:
TExtract.head()

In [ ]:
TExtract['TextClean'] = TExtract['text']

In [ ]:
# remove '\n', lowercase all letters
TExtract['TextClean'] = TExtract['TextClean'].apply(lambda x: x.replace('\n',' ').lower())

In [ ]:
TExtract[['text', 'TextClean']].head()

In [ ]:
TExtract.head()

In [ ]:
#Remove user handles from tweets
TExtract['TextClean'] = TExtract['TextClean'].str.replace('(\@\w+.*?)'," ",regex=True)

In [ ]:
# expand contractions
TExtract['TextClean'] = TExtract['TextClean'].apply(lambda x: contractions.fix(x))

# remove punctuations
TExtract['TextClean'] = TExtract['TextClean'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))

#remove consecutive characters that occur three or more times in a row, and replace them with just two occurrences of that character.
TExtract['TextClean'] = TExtract['TextClean'].apply(lambda x: re.compile(r"(.)\1{2,}").sub(r"\1\1", x))

# Removing extra spaces
TExtract['TextClean'] = TExtract['TextClean'].apply(lambda x: re.sub(' +',' ',x))

Removing stop words
stop_words = set(stopwords.words('english'))
TExtract['TextClean'] = TExtract['TextClean'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

In [ ]:
TExtract[['text', 'TextClean']].head()

In [ ]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Create a SentimentIntensityAnalyzer object.
sid_obj = SentimentIntensityAnalyzer()

# function to print sentiments of the sentence.
def sentiment_scores(sentence):
    # polarity_scores method of SentimentIntensityAnalyzer object
    # gives a sentiment dictionary, which contains pos, neg, neu, and
    # compound scores.
    sentiment_dict = sid_obj.polarity_scores(sentence)

    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05:
        return "positive"
    elif sentiment_dict['compound'] <= -0.05:
        return "negative"
    else:
        return "neutral"

In [ ]:
# Load the model
model = tweetnlp.load_model('sentiment')

# Define the sentiment analysis function
def tweetnlp_sentiment(text):
    return model.sentiment(text)['label']

In [ ]:
# apply the sentiment_scores function to the 'text' column
TExtract['VaderSentiment(Raw)'] = TExtract['text'].apply(lambda x: sentiment_scores(x))

In [ ]:
# Apply the function to each row of the 'TExtract[PreProcessedText]' column and store the result in a new column called 'sentiment'
TExtract['TweetNLPsentiment(Raw)'] = TExtract['text'].apply(tweetnlp_sentiment)

In [ ]:
# apply the sentiment_scores function to the 'TextClean' column
TExtract['VaderSentiment(Clean)'] = TExtract['TextClean'].apply(lambda x: sentiment_scores(x))

In [ ]:
# Apply the function to each row of the 'TExtract[textclean]' column and store the result in a new column called 'sentiment'
TExtract['TweetNLPsentiment(Clean)'] = TExtract['TextClean'].apply(tweetnlp_sentiment)

In [ ]:
TExtract[["HumanSentiment", "VaderSentiment(Raw)", "TweetNLPsentiment(Raw)", "VaderSentiment(Clean)", "TweetNLPsentiment(Clean)"]].apply(pd.Series.value_counts)

In [ ]:
# create a dataframe for the counts of each sentiment value
sentiment_counts = TExtract[["HumanSentiment", "VaderSentiment(Raw)", "TweetNLPsentiment(Raw)", "VaderSentiment(Clean)", "TweetNLPsentiment(Clean)"]].apply(pd.Series.value_counts)

# plot the bar chart
sentiment_counts.plot.bar(rot=0, figsize=(10,6))
plt.title('Sentiment Analysis Prediction Comparison')
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.legend(title='Classifier', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [ ]:
y_test = TExtract['HumanSentiment']

In [ ]:
class_names=['negative','neutral','positive']

In [ ]:
VaderSentiment_pred = TExtract['VaderSentiment(Raw)']

# calculate evaluation metrics
VaderSentiment_accuracy = accuracy_score(y_test, VaderSentiment_pred)
VaderSentiment_precision = precision_score(y_test, VaderSentiment_pred, average='macro')
VaderSentiment_recall = recall_score(y_test, VaderSentiment_pred, average='macro')
VaderSentiment_f1 = f1_score(y_test, VaderSentiment_pred, average='macro')
VaderSentiment_confusion_mat = confusion_matrix(y_test, VaderSentiment_pred)

print('VaderSentiment_accuracy: ',  VaderSentiment_accuracy*100)
print('VaderSentiment_precision: ',  VaderSentiment_precision*100)
print('VaderSentiment_recall: ',  VaderSentiment_recall*100)
print('VaderSentiment_f1: ',  VaderSentiment_f1*100)
print('VaderSentiment_confusion_mat: \n',  VaderSentiment_confusion_mat)

In [ ]:
VaderSentiment_class_report=classification_report(y_test, VaderSentiment_pred, target_names=class_names)
print(VaderSentiment_class_report)

In [ ]:
VaderSentimentClean_pred = TExtract['VaderSentiment(Clean)']

# calculate evaluation metrics
VaderSentimentClean_accuracy = accuracy_score(y_test, VaderSentimentClean_pred)
VaderSentimentClean_precision = precision_score(y_test, VaderSentimentClean_pred, average='macro')
VaderSentimentClean_recall = recall_score(y_test, VaderSentimentClean_pred, average='macro')
VaderSentimentClean_f1 = f1_score(y_test, VaderSentimentClean_pred, average='macro')
VaderSentimentClean_confusion_mat = confusion_matrix(y_test, VaderSentimentClean_pred)

print('VaderSentimentClean_accuracy: ',  VaderSentimentClean_accuracy*100)
print('VaderSentimentClean_precision: ',  VaderSentimentClean_precision*100)
print('VaderSentimentClean_recall: ',  VaderSentimentClean_recall*100)
print('VaderSentimentClean_f1: ',  VaderSentimentClean_f1*100)
print('VaderSentimentClean_confusion_mat: \n',  VaderSentimentClean_confusion_mat)

In [ ]:
VaderSentimentClean_class_report=classification_report(y_test, VaderSentimentClean_pred, target_names=class_names)
print(VaderSentimentClean_class_report)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# plot confusion matrix
sns.set(font_scale=1.4) # adjust font size
sns.heatmap(VaderSentimentClean_confusion_mat, annot=True, annot_kws={"size": 16}, cmap='Oranges', fmt='g',
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
plt.ylabel('True label')
plt.xlabel('Vader Predicted label')
plt.title('Vader Sentiment Analysis Confusion Matrix (Clean Text)')
plt.show()

In [ ]:
# plot confusion matrix
sns.set(font_scale=1.4) # adjust font size
sns.heatmap(VaderSentiment_confusion_mat, annot=True, annot_kws={"size": 16}, cmap='Oranges', fmt='g',
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
plt.ylabel('True label')
plt.xlabel('Vader Predicted label')
plt.title('Vader Sentiment Analysis Confusion Matrix (Raw Text)')
plt.show()

In [ ]:
TweetNLPsentiment_pred = TExtract['TweetNLPsentiment(Raw)']

# calculate evaluation metrics
TweetNLPsentiment_accuracy = accuracy_score(y_test, TweetNLPsentiment_pred)
TweetNLPsentiment_precision = precision_score(y_test, TweetNLPsentiment_pred, average='macro')
TweetNLPsentiment_recall = recall_score(y_test, TweetNLPsentiment_pred, average='macro')
TweetNLPsentiment_f1 = f1_score(y_test, TweetNLPsentiment_pred, average='macro')
TweetNLPsentiment_confusion_mat = confusion_matrix(y_test, TweetNLPsentiment_pred)

print('TweetNLPsentiment_accuracy: ',  TweetNLPsentiment_accuracy*100)
print('TweetNLPsentiment_precision: ',  TweetNLPsentiment_precision*100)
print('TweetNLPsentiment_recall: ',  TweetNLPsentiment_recall*100)
print('TweetNLPsentiment_f1: ',  TweetNLPsentiment_f1*100)
print('TweetNLPsentiment_confusion_mat: \n',  TweetNLPsentiment_confusion_mat)

In [ ]:
TweetNLPsentiment_class_report=classification_report(y_test, TweetNLPsentiment_pred, target_names=class_names)
print(TweetNLPsentiment_class_report)

In [ ]:
TweetNLPsentimentClean_pred = TExtract['TweetNLPsentiment(Clean)']

# calculate evaluation metrics
TweetNLPsentimentClean_accuracy = accuracy_score(y_test, TweetNLPsentimentClean_pred)
TweetNLPsentimentClean_precision = precision_score(y_test, TweetNLPsentimentClean_pred, average='macro')
TweetNLPsentimentClean_recall = recall_score(y_test, TweetNLPsentimentClean_pred, average='macro')
TweetNLPsentimentClean_f1 = f1_score(y_test, TweetNLPsentimentClean_pred, average='macro')
TweetNLPsentimentClean_confusion_mat = confusion_matrix(y_test, TweetNLPsentimentClean_pred)

print('TweetNLPsentimentClean_accuracy: ',  TweetNLPsentimentClean_accuracy*100)
print('TweetNLPsentimentClean_precision: ',  TweetNLPsentimentClean_precision*100)
print('TweetNLPsentimentClean_recall: ',  TweetNLPsentimentClean_recall*100)
print('TweetNLPsentimentClean_f1: ',  TweetNLPsentimentClean_f1*100)
print('TweetNLPsentimentClean_confusion_mat: \n',  TweetNLPsentimentClean_confusion_mat)

In [ ]:
TweetNLPsentimentClean_class_report=classification_report(y_test, TweetNLPsentimentClean_pred, target_names=class_names)
print(TweetNLPsentimentClean_class_report)

In [ ]:
# plot confusion matrix
sns.set(font_scale=1.4) # adjust font size
sns.heatmap(TweetNLPsentimentClean_confusion_mat, annot=True, annot_kws={"size": 16}, cmap='Greens', fmt='g',
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
plt.ylabel('True label')
plt.xlabel('TweetNLP Predicted label')
plt.title('TweetNLP Sentiment Analysis Confusion Matrix (Clean Text)')
plt.show()

In [ ]:
# plot confusion matrix
sns.set(font_scale=1.4) # adjust font size
sns.heatmap(TweetNLPsentiment_confusion_mat, annot=True, annot_kws={"size": 16}, cmap='Greens', fmt='g',
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
plt.ylabel('True label')
plt.xlabel('TweetNLP Predicted label')
plt.title('TweetNLP Sentiment Analysis Confusion Matrix (Raw Text)')
plt.show()

In [ ]:
# Define a function to apply sentiment analysis to each tweet text
def analyze_sentiment(text):
    result = model.sentiment(text, return_probability=True)
    max_prob_key = max(result['probability'], key=result['probability'].get)
    return pd.Series({'Sentiment': result['label'], 
                      'Probability': result['probability'][max_prob_key]})

# Apply the function to the 'TextClean' column and store the result in two new columns
TExtract[['Sentiment', 'Probability']] = TExtract['text'].apply(analyze_sentiment)

In [ ]:
# Plot histogram with density line
plt.hist(TExtract['Probability'], bins=20, density=True, alpha=0.7, color = 'darkgreen')
TExtract['Probability'].plot(kind='kde', color='r')
plt.xlim(0.2, 1.2)
# Add labels
plt.title('Density Plot of TweetNLP Sentiment Probability')
plt.xlabel('Probability')
plt.ylabel('Density')
plt.show()

In [ ]:
# create a dataframe for the counts of each sentiment value
sentiment_counts = TExtract[["HumanSentiment", "Sentiment"]].apply(pd.Series.value_counts)

# plot the bar chart
ax = sentiment_counts.plot.bar(rot=0, figsize=(10,6))
ax.set_title('Human vs TweetNLP Classification Comparison')
ax.set_xlabel('Sentiment')
ax.set_ylabel('Count')
ax.legend(labels=['Human', 'TweetNLP'], title='Classifier', bbox_to_anchor=(1, 1))
plt.show()

In [ ]:
TExtract[['text', 'Sentiment', 'Probability']].head()

In [ ]:
TExtract[['text', 'Sentiment', 'Probability']].sort_values('Probability', ascending=True).head(5)


In [ ]:
TExtract[['text', 'Sentiment', 'Probability']].loc[TExtract['Probability'] < 0.5].count()


In [ ]:
TExtract[['text', 'Sentiment', 'Probability']].loc[TExtract['Probability'] >= 0.5].count()


In [ ]:
#Filter out tweets where the sentiment classification probability is less than 0.5
tweets_df = TExtract.loc[TExtract['Probability'] > 0.5]

In [ ]:
y_test = tweets_df['HumanSentiment']
TweetNLPsentiment_pred = tweets_df['TweetNLPsentiment(Raw)']

# calculate evaluation metrics
TweetNLPsentiment_accuracy = accuracy_score(y_test, TweetNLPsentiment_pred)
TweetNLPsentiment_precision = precision_score(y_test, TweetNLPsentiment_pred, average='macro')
TweetNLPsentiment_recall = recall_score(y_test, TweetNLPsentiment_pred, average='macro')
TweetNLPsentiment_f1 = f1_score(y_test, TweetNLPsentiment_pred, average='macro')
TweetNLPsentiment_confusion_mat = confusion_matrix(y_test, TweetNLPsentiment_pred)

print('TweetNLPsentiment_accuracy: ',  TweetNLPsentiment_accuracy*100)
print('TweetNLPsentiment_precision: ',  TweetNLPsentiment_precision*100)
print('TweetNLPsentiment_recall: ',  TweetNLPsentiment_recall*100)
print('TweetNLPsentiment_f1: ',  TweetNLPsentiment_f1*100)
print('TweetNLPsentiment_confusion_mat: \n',  TweetNLPsentiment_confusion_mat)

In [ ]:
# plot confusion matrix
sns.set(font_scale=1.4) # adjust font size
sns.heatmap(TweetNLPsentiment_confusion_mat, annot=True, annot_kws={"size": 16}, cmap='Greens', fmt='g',
            xticklabels=['Negative', 'Neutral', 'Positive'],
            yticklabels=['Negative', 'Neutral', 'Positive'])
plt.ylabel('True label')
plt.xlabel('TweetNLP Predicted label')
plt.title('TweetNLP Sentiment Analysis Confusion Matrix')
plt.show()

In [12]:
topic = 'ULEZ'
location = 'London'
start_date = '2023-05-25'
end_date = '2023-06-01'
max_tweets = 100

In [3]:
topic2 = topic.replace(',', " OR ")
topic2 = topic2.replace('  ', ' ')

In [8]:
# Convert strings to dates
start_date = datetime.strptime(start_date, '%Y-%m-%d')
end_date = datetime.strptime(end_date, '%Y-%m-%d')

In [14]:

query = f'{topic}) near:"{location}" within:300km lang:en since:{period_start.strftime("%Y-%m-%d")} until:{period_end.strftime("%Y-%m-%d")} -filter:links -filter:retweet'
        for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
            time.sleep(1) # add a time delay of 1 second
            if i > Qt_tweets:
                break
            tweets_list.append([tweet.date, tweet.rawContent, tweet.user.username, tweet.viewCount])
# Create a Pandas DataFrame from the list of tweets
tweets_df = pd.DataFrame(tweets_list, columns=['Date', 'Text', 'Username', 'Views'])


IndentationError: unexpected indent (34677350.py, line 2)